In [91]:
using LinearAlgebra
using SparseArrays
using Test

In [123]:
function get_unique_elements(x_1::Vector, x_2::Vector=[])
    
    """
    Finds indices of unique elements in Array x.
    
    # Arguments
    - 'x_1::Vector{Any}': object of which to find unique indices
    - 'x_2::Vector{Any}': Array with same length as x_1, Optional (Default is [])
    
    # Returns
    - 'x_1_unique::Vector{Any}': Array of unique entries in x_1
    - 'x_2_unique::Vector{Any}': Array of entries corresponding to unique elements in x_1
    - 'unique_indices::Vector{Int}': Array with positions of unique elements in x
    """
    sorted_x_1, sorted_x_2, sorted_list = deg_lex_sort(x_1, x_2)
    unique_indices = unique(i -> sorted_x_1[i], 1:length(sorted_x_1))
    x_1_unique = sorted_x_1[unique_indices]
    if x_2 != []
        x_2_unique = sorted_x_2[unique_indices]
    else
        x_2_unique = x_2
    end
        
    return x_1_unique, x_2_unique, unique_indices
    
end

get_unique_elements (generic function with 2 methods)

In [109]:
function mat_to_arr_of_arrs(A::Matrix{Any}, col_is_row::Int64=0)
    """
    Transforms an object of type Matrix{Any} to an Array of Arrays where each row is an individual Array.
    
    # Arguments
    - 'A::Matrix{Any}': the matrix to transform
    - 'col_is_row::Int': If 1 (default 0), instead the columns of the matrix become individual Arrays.
    
    # Returns
    - 'transformed_A::Vector{Vector{Any}}': transformed Array
    """

    if  col_is_row == 0
        # converts matrix to array of arrays with rows of matrix being the individual entries of array,
        # i.e. first row of matrix A becomes the array result[1]
        transformed_A = [A[i,:] for i in 1:size(A,1)]
    
    
    elseif col_is_row == 1
        # converts matrix to array of arrays with columns of matrix being the individual entries of array,
        # i.e. first column of matrix A becomes the array result[1]
        transformed_A = [A[:,i] for i in 1:size(A,2)]


    else
        println("The optional argument 'col_is_row' was given an invalid value.")
        return 
        
    end
    
    return transformed_A
    
end

mat_to_arr_of_arrs (generic function with 2 methods)

In [110]:
function construct_border(
        terms::Vector{Vector{Int64}}, terms_evaluated::Vector{Vector{Float64}}, X_train::Vector{Vector{Float64}}, 
        degree_1_terms::Vector{Vector{Int64}}=[],  
        degree_1_terms_evaluated::Vector{Vector{Float64}}=[], 
        purging_terms::Vector{Vector{Int64}}=[])
    """
    Constructs the border of terms. 
    
    # Arguments
    - 'terms::Vector{Vector{Int}}': Array of terms to construct border from
    - 'terms_evaluated::Vector{Vector{Float64}}': Array of evaluations of terms
    - 'X_train::Vector{Vector{Float64}}': Array of n-dimensional data points
    - 'degree_1_terms::Vector{Vector{Int64}}': Array of terms of degree 1 to construct border terms, Optional 
        (Default is [])
    - 'degree_1_terms_evaluated::Vector{Float64}': Array of evaluations of degree_1_terms, Optional 
        (Default is [])
    - 'purging_terms::Vector{Vector{Int64}}': Purge terms divisible by these terms, Optional 
        (Default is [])
    
    # Returns
    - 'border_terms_raw::Vector{Vector{Int64}}': Array of unpurged border terms
    - 'border_evaluations_raw::Vector{Vector{Float64}}': Array of evaluations of border terms
    - 'non_purging_indices::Vector{Int64}': Array unique and non-purging indices in border_terms_raw
    """
    
    if degree_1_terms == []
        # 'I' comes from the LinearAlgebra package
        border_terms_raw = mat_to_arr_of_arrs(
            1 * Matrix(I, length(X_train[1]), length(X_train[1])))
            
        border_terms_raw_evaluation = X_train
        
    else
        # terms repeat + tile part
        terms_repeat = repeat(terms, inner=length(degree_1_terms))    
        l_terms = length(terms_repeat)
        m_terms = length(degree_1_terms)
        k_terms = Int(l_terms / m_terms) 
        degree_1_terms_tile = repeat(degree_1_terms, outer=k_terms)        
        border_terms_raw = degree_1_terms_tile + terms_repeat
        
        # evaluation repeat + tile part
        terms_evaluated_repeat = repeat(terms_evaluated, inner=length(degree_1_terms_evaluated))
        l_eval = length(terms_evaluated_repeat)
        m_eval = length(degree_1_terms_evaluated)
        k_eval = Int(l_eval / m_eval)
        degree_1_terms_evaluated_tile = repeat(degree_1_terms_evaluated, outer=k_eval)
        border_evaluations_raw = [degree_1_terms_evaluated_tile[i] .* terms_evaluated_repeat[i] for i in 1:l_eval]
         
    end
    
    border_terms_purged, border_evaluations_purged, unique_indices = get_unique_elements(
        border_terms_raw, border_evaluations_raw)
    
    if purging_terms != []
        border_terms_purged, border_evaluations_purged, unique_indices_2 = purge(
            border_terms_purged, border_evaluations_purged, purging_terms)
        
        if unique_indices_2 != []
            non_purging_indices = [unique_indices[i] for i in unique_indices_2]
        else
            non_purging_indices = unique_indices
        end
    else
        non_purging_indices = unique_indices
    end
    
    return border_terms_raw, border_evaluations_raw, non_purging_indices
    
end

construct_border (generic function with 4 methods)

In [111]:
function purge(
        terms::Vector{Vector{Int64}}, 
        terms_evaluated::Vector{Vector{Float64}}, 
        purging_terms::Vector{Vector{Int64}})
    """
    Purges all the terms in 'terms' that are divisible by at least one term in 'purging_terms'.
    
    # Arguments
    - 'terms::Vector{Vector{Int64}}': Array of terms 
    - 'terms_evaluated:::Vector{Vector{Float64}}': Array of evaluations of terms 
    - 'purging_terms::Vector{Vector{Int64}}':  Array of possible divisor terms
    
    # Returns
    - 'terms[indices]::Vector{Vector{Int64}}': purged version of terms
    - 'terms_evaluated[indices]::Vector{Vector{Float64}}': purged version of terms_evaluated
    - 'indices::Vector{Int64}': Array of indices of purged terms in 'terms'
    """
    indices = [x for x in 1:length(terms)]
    purge_indices = []
    for i in eachindex(terms)
        for j in eachindex(purging_terms)
            if all(terms[i] .- purging_terms[j] .>= 0)
                append!(purge_indices, i)
                break
            end
        end
    end
    indices = deleteat!(indices, purge_indices)
    return terms[indices], terms_evaluated[indices], indices
end

purge (generic function with 1 method)

In [113]:
@testset "Test suite for purge" begin
    matrix_2 = [
        [1, 0, 1],
        [2, 0, 1],
        [3, 1, 1]
    ]
    matrix_1 = [
        [1, 0, 0],
        [1, 1, 1]
    ]
    matrix_2_purged, matrix_2_purged_2, _ = purge(matrix_2, 1.0*matrix_2, matrix_1)
    @test length(matrix_2_purged) == 0
    @test length(matrix_2_purged_2) == 0
    
    matrix_2 = [
        [1, 0, 1],
        [2, 0, 1],
        [3, 1, 1]
    ]
    matrix_1 = [
        [1, 0, 2],
        [1, 2, 1]
    ]
    matrix_2_purged, matrix_2_purged_2, _ = purge(matrix_2, 1.0*matrix_2, matrix_1)
    @test matrix_2_purged == matrix_2_purged_2
    @test matrix_2_purged == [[1, 0, 1], [2, 0, 1], [3, 1, 1]]
end

Test Summary:        | Pass  Total  Time
Test suite for purge |    4      4  0.0s


Test.DefaultTestSet("Test suite for purge", Any[], 4, false, false, true, 1.684423543142e9, 1.684423543142e9)

In [114]:
function monomial_evaluation(m::Vector{Int64}, x::Vector{Float64}, coeff=1)
    """
    Evaluates monomial m at point x.
    
    # Arguments
    - 'm::Vector{Int64}': monomial under which to evaluate x
    - 'x::Vector{Float64}': point at which to evaluate m
    - 'coeff=1': coefficient for monomial m, Optional
    
    # Returns 
    - 'evaluation::Float64': evaluation of m at point x
    """
    result = 1
    
    for i in 1:length(m)
        result *= x[i]^m[i]
    end
    
    evaluation = coeff * result
    return evaluation
end

monomial_evaluation (generic function with 2 methods)

In [115]:
function monomial_evaluation_set(m::Vector{Int64}, X::Vector{Vector{Float64}}, coeff=1)
    """
    Evaluates a set of points X under a single monomial m.
    
    # Arguments
    - 'm::Vector{Int64}': monomial under which to evaluate X
    - 'X::Vector{Vector{Float64}}': set of points to evaluate
    - 'coeff=1': coefficient for monomial m, Optional
    
    # Returns
    'evaluated::Vector{Float64}': Array of evaluations of m, evaluated[i] is the evaluation of m at point X[i].
    """    
    results = ones(length(X))
        
    for i in 1:length(X)
        results[i] = monomial_evaluation(m, X[i])
    end
    evaluated = coeff * results
    return evaluated
    
end

monomial_evaluation_set (generic function with 2 methods)

In [116]:
function monomial_set_evaluation_set(M::Vector{Vector{Int64}}, X::Vector{Vector{Float64}}, coeffs=1)
    """
    Evaluates a set of points X under a set of monomials M.
    
    # Arguments
    - 'M::Vector{Vector{Int64}}': set of monomials under which to evaluate X
    - 'X::Vector{Vector{Float64}}': set of points at which to evaluate M
    - 'coeffs=1': coefficient(s) for all (each) monomial in M, Optional
    
    # Returns
    - 'all_evaluated::Vector{Vector{Float64}}': Array of Arrays where all_evaluated[i][j] is 
    monomial M[i] evaluated at point X[j]
    """        
    result = [[] for _ in 1:length(M)]
    
    for i in 1:length(M)
        result[i] = monomial_evaluation_set(M[i], X)
    end
    
    all_evaluated = coeffs .* result
    return all_evaluated
    
end

monomial_set_evaluation_set (generic function with 2 methods)

In [117]:
@testset "Test suite for monomial_evaluation_set" begin
    m1 = [4, 1, 2]
    X1 = [[1.0, 2.0, 3.0], [1.0, 0.5, 2.0]]
    coeff = 4.5
    @test monomial_evaluation_set(m1, X1) == [18.0, 2.0]
    @test monomial_evaluation_set(m1, X1, coeff) == [81.0, 9.0]
end

Test Summary:                          | Pass  Total  Time
Test suite for monomial_evaluation_set |    2      2  0.2s


Test.DefaultTestSet("Test suite for monomial_evaluation_set", Any[], 2, false, false, true, 1.684423544714e9, 1.68442354493e9)

In [118]:
@testset "Test suite for monomial_set_evaluation_set" begin
    M1 = [[4, 1, 2], [3, 0, 2], [0, 2, 1]]
    X1 = [[1.0, 2.0, 3.0], [1.0, 0.5, 2.0]]
    coeffs = [2.0, 1.0, 5.0]
    @test monomial_set_evaluation_set(M1, X1) == [[18.0, 2.0], [9.0, 4.0], [12.0, 0.5]]
    @test monomial_set_evaluation_set(M1, X1, coeffs) == [[36.0, 4.0], [9.0, 4.0], [60.0, 2.5]]
end

Test Summary:                              | Pass  Total  Time
Test suite for monomial_set_evaluation_set |    2      2  0.5s


Test.DefaultTestSet("Test suite for monomial_set_evaluation_set", Any[], 2, false, false, true, 1.684423544936e9, 1.684423545466e9)

In [119]:
function compute_degree(matrix::Vector{Vector{Int64}})
    """
    Computes the sum of the row entries, which for matrices representing terms is equal to the degree.
    
    # Arguments
    - 'matrix::Vector{Vector{Int64}}': matrix whose row degrees have to be computed.
    
    # Returns
    - 'degree_array::Vector{Int64}': Array containing degrees of all rows.
    """
    degree_array = sum.(matrix)
    return degree_array
end

compute_degree (generic function with 1 method)

In [120]:
function deg_lex_sort(matrix_1::Vector{Vector{Int64}}, matrix_2=[])
    """
    Sorts the rows of matrix_1 degree-lexicographically and matrix_2 accordingly
    
    # Arguments
    - 'matrix_1::Vector{Vector{Int64}}': term matrix to be sorted
    - 'matrix_2::Vector{Vector{Int64}}': matrix getting sorted the same way matrix_1 is (Default is [])
    
    # Returns
    - 'matrix_1::Vector{Vector{Int64}}': matrix_1 sorted degree_lexicographically
    - 'matrix_2::Vector{Vector{Int64}}': matrix_2 sorted like matrix_1
    """
    degrees = compute_degree(matrix_1)
    for i in 1:length(degrees)
        # using reverse + Base.sortperm sorts first by degree and then 
        # from last to first variable exponent.
        matrix_1[i] = append!([degrees[i]], reverse(matrix_1[i]))
    end
    # find the permutation to correctly sort matrix_1
    sorted_list = sortperm(matrix_1)
    matrix_1 = matrix_1[sorted_list]
    # deletes first entry (degree) in each term
    matrix_1 = [reverse(deleteat!(matrix_1[i], 1)) for i in 1:length(matrix_1)]
    if matrix_2 != []
        matrix_2 = matrix_2[sorted_list]
    end
    return matrix_1, matrix_2, sorted_list
end

deg_lex_sort (generic function with 3 methods)

In [121]:
matrix = [
    [1,0,1],
    [2,1,3],
    [3,2,1],
    [4,3,2],
    [3,2,1]
]
matrix_sorted = [
    [1,0,1],
    [3,2,1],
    [3,2,1],
    [2,1,3],
    [4,3,2]
]
matrix_sorted_unique = [
    [1,0,1],
    [3,2,1],
    [2,1,3],
    [4,3,2]
]

4-element Vector{Vector{Int64}}:
 [1, 0, 1]
 [3, 2, 1]
 [2, 1, 3]
 [4, 3, 2]

In [122]:
@testset "Test suite for deg_lex_sort" begin
    """
    Tests whether deg_lex_sort behaves as intended.
    """
    # since deg_lex_sort directly alters the input matrix, need to pass a copy of 'matrix' to ensure 
    # matrix_2 permutates an unchanged copy of 'matrix'
    matrix_1, matrix_2 = copy(matrix), copy(matrix)
    matrix_sorted_1, matrix_sorted_2, a = deg_lex_sort(matrix_1, matrix_2)
    @test matrix_sorted_1 == matrix_sorted
    @test matrix_sorted_2 == matrix_sorted
end

Test Summary:               | Pass  Total  Time
Test suite for deg_lex_sort |    2      2  0.0s


Test.DefaultTestSet("Test suite for deg_lex_sort", Any[], 2, false, false, true, 1.68442354681e9, 1.684423546859e9)

In [107]:
@testset "Test suite for compute_degree" begin
    """
    Tests whether compute_degree behaves as intended.
    """
    matrix_1 = copy(matrix)
    @test compute_degree(matrix) == [2, 6, 6, 9, 6]
end

Test Summary:                 | Pass  Total  Time
Test suite for compute_degree |    1      1  0.0s


Test.DefaultTestSet("Test suite for compute_degree", Any[], 1, false, false, true, 1.684423385646e9, 1.684423385646e9)

In [124]:
@testset "Test suite for get_unique_elements" begin
    """
    Tests whether get_unique_elements behaves as intended.
    """
    matrix_1, matrix_2 = copy(matrix), copy(matrix)
    matrix_unique_1, matrix_unique_2, _ = get_unique_elements(matrix_1, matrix_2)
    @test matrix_unique_1 == matrix_sorted_unique
    @test matrix_unique_2 == matrix_sorted_unique
end

Test Summary:                      | Pass  Total  Time
Test suite for get_unique_elements |    2      2  0.3s


Test.DefaultTestSet("Test suite for get_unique_elements", Any[], 2, false, false, true, 1.684423753148e9, 1.684423753483e9)